# Modeling

### Importing necessary libraries

In [1]:
import json
import string
import random
import numpy as np
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier  # Add this import statement
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pennygituku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Converting json into a dataframe

In [2]:
# Load your dataset from a JSON file
file_path = '../Final_Intents.json'

with open(file_path, 'r') as file:
    data = json.load(file)

### Pre-procassing Data

In [3]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Your file path
file_path = '../Final_Intents.json'

# Provided tags
tags = [
    'Moringa_School_News',
    'Cybersecurity_Certifications',
    'Enrollment_Information',
    'Moringa_School_Testimonials',
    'Moringa_School_Courses',
    'UI_UX_Design',
    'Software_Engineering_Mobile_Track',
    'Quality_Assurance_Testers',
]

# Initialize lists
words = []
classes = tags  
data_X = []
data_Y = []

# Read and process the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

stop_words = set(stopwords.words('english'))

# Iterating over all intents in the loaded data
for intent in data:
    patterns = intent.get("questions", [])  
    tag = intent.get("tag", "")

    for pattern in patterns:
        # Tokenize and preprocess the pattern
        pattern_tokens = [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(pattern) if word.isalnum() and word not in string.punctuation and word not in stop_words]

        # Extend words list with tokens from the pattern
        words.extend(pattern_tokens)

        # Append the preprocessed pattern and corresponding tag to data_X and data_Y
        data_X.append(pattern_tokens)
        data_Y.append(tag)

        # Adding the tag to the classes 
        if tag not in classes:
            classes.append(tag)

# Lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [word.lower() for word in words if word not in string.punctuation]

# Removing stop words
words = [word for word in words if word not in stop_words]

# Sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

print("Classes:", classes)
print("Example Pattern:", data_X[0] if data_X else "No patterns available")
print("Example Tag:", data_Y[0] if data_Y else "No tags available")

Classes: ['Advanced_Data_Science_Topics', 'Contact', 'Cybersecurity_Career_Opportunities', 'Cybersecurity_Certifications', 'Cybersecurity_Course_Information', 'Data_Ethics_and_Privacy', 'Data_Science_Admission_Eligibility_Criteria', 'Data_Science_Application_Process', 'Data_Science_Applications_in_Business', 'Data_Science_Career', 'Data_Science_Career_Opportunities', 'Data_Science_Career_Transition_Support', 'Data_Science_Certification_and_Accreditation', 'Data_Science_Collaborative_Projects_and_Teamwork', 'Data_Science_Competitions_and_Challenges', 'Data_Science_Course_Duration_and_Fees', 'Data_Science_Course_Overview', 'Data_Science_Course_Pacing_and_Flexibility', 'Data_Science_Course_Schedule_and_Structure', 'Data_Science_Courses', 'Data_Science_Curriculum_Outline', 'Data_Science_Cybersecurity_Introduction', 'Data_Science_Industry_Relevance', 'Data_Science_Networking_and_Collaboration_Opportunities', 'Data_Science_Online_and_Offline_Learning_Modes', 'Data_Science_Pre-Work_Requiremen

## Modeling

### Splitting data

In [4]:
# train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.2, random_state=42)

In [5]:
# Convert tokenized patterns back to strings

X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

In [6]:
#CountVectorizer

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform([" ".join(pattern) for pattern in X_train])
X_test_vectorized = vectorizer.transform([" ".join(pattern) for pattern in X_test])

### Multinomial Naive Bayes

In [7]:
# MultinomialNB

model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

In [8]:
# Model accuracy
accuracy = model.score(X_test_vectorized, y_test)
print("Model Accuracy:", accuracy)

# Classification report
y_pred = model.predict(X_test_vectorized)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Model Accuracy: 0.25190839694656486
Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       0.00      0.00      0.00         2
                     Cybersecurity_Career_Opportunities       0.00      0.00      0.00         3
                           Cybersecurity_Certifications       0.33      1.00      0.50         1
                       Cybersecurity_Course_Information       0.00      0.00      0.00         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                                    Data_Science_Career       0.00      0.00      0.00         2
                      Data_Science_Career_Opportunities       0.13

The model exhibits diverse performance across classes. Notably:

1. **High Precision, Low Recall:**
   - `Cybersecurity_Certifications`: Precision 33%, Recall 100%
   - `Data_Science_Career_Opportunities`: Precision 13%, Recall 50%
   - `Data_Science_Online_and_Offline_Learning_Modes`: Precision 14%, Recall 100%
   - `Software_Engineering_Mobile_Track`: Precision and Recall both 100%
   - `Financial Aid`: Precision and Recall both 67%
   - `cloud_computing`: Precision and Recall both 100%
   - `cloud_developer_role`: Precision and Recall both 100%
   - `hr_data_analytics`: Precision and Recall both 100%

2. **Other Classes:**
   - `Event`: Precision, Recall, and F1-score all 100%
   - `Learning_Resources`: Low performance with Precision, Recall, and F1-score all 0%

In summary, precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The mentioned classes may benefit from specific adjustments to enhance model performance.

### GridSearchCV Multinomial Naive Bayes

In [9]:
# Define the parameter grid to search
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Unigrams or bigrams
    'clf__alpha': [0.1, 0.5, 1.0, 1.5, 2.0]
}

# Create a pipeline with CountVectorizer and Multinomial Naive Bayes
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# Create a GridSearchCV object
grid_search = GridSearchCV(text_clf, param_grid, cv=5)
grid_search.fit(X_train_str, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = grid_search.best_estimator_

# Evaluate the best model
best_accuracy = best_model.score(X_test_str, y_test)
print("Best Model Accuracy:", best_accuracy)

# Classification report
y_pred_best_model = best_model.predict(X_test_str)
print("Classification Report:\n", classification_report(y_test, y_pred_best_model))

# Confusion matrix
conf_matrix_best_model = confusion_matrix(y_test, y_pred_best_model)
print("Confusion Matrix:\n", conf_matrix_best_model)

Best Model Accuracy: 0.2900763358778626
Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       1.00      0.50      0.67         2
                     Cybersecurity_Career_Opportunities       0.00      0.00      0.00         3
                           Cybersecurity_Certifications       0.25      1.00      0.40         1
                       Cybersecurity_Course_Information       1.00      0.33      0.50         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                                    Data_Science_Career       0.25      0.50      0.33         2
                      Data_Science_Career_Opportunities       

The model achieved an accuracy of 29.01%, with diverse precision, recall, and F1-score values across different classes. Notable observations include:

1. **High Precision, Low Recall:**
   - `Cybersecurity_Certifications`: Precision 25%, Recall 100%
   - `Data_Science_Career_Opportunities`: Precision 20%, Recall 50%
   - `Data_Science_Online_and_Offline_Learning_Modes`: Recall 100%
   - `Software_Engineering_Mobile_Track`: Precision and Recall both 100%
   - `Financial Aid`: Precision and Recall both 67%
   - `cloud_computing`: Precision and Recall both 100%
   - `cloud_developer_role`: Precision and Recall both 100%
   - `hr_data_analytics`: Precision and Recall both 100%
   - `aws_cloud_course`, `aws_cloud_practitioner`, `career_services`, `cloud_computing`, `cloud_developer_role`: Precision and Recall all 100%

2. **Other Classes:**
   - `Event`: Precision, Recall, and F1-score all 100%
   - `Learning_Resources`: Low performance with Precision, Recall, and F1-score all 0%
   - `enrollment_deadline`: Precision 33%, Recall 100%

In summary, the model's performance across classes remains diverse. Precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides insights into correct and incorrect predictions for each class, aiding in identifying areas for improvement. Further refinement through hyperparameter tuning or additional data may enhance overall performance.

### TF-IDF vectorizer and SVM classifier

In [10]:
# Convert tokenized patterns back to strings
X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

# Create a pipeline with TF-IDF vectorizer and SVM classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1.0))  # You can experiment with different kernel and C values
])

# Train the model
pipeline.fit(X_train_str, y_train)

# Evaluate the model
accuracy_tfidf_svm = pipeline.score(X_test_str, y_test)
print("TF-IDF + SVM Model Accuracy:", accuracy_tfidf_svm)

# Classification report
y_pred_tfidf_svm = pipeline.predict(X_test_str)
print("Classification Report:\n", classification_report(y_test, y_pred_tfidf_svm))

# Confusion matrix
conf_matrix_tfidf_svm = confusion_matrix(y_test, y_pred_tfidf_svm)
print("Confusion Matrix:\n", conf_matrix_tfidf_svm)

TF-IDF + SVM Model Accuracy: 0.3969465648854962
Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       1.00      0.50      0.67         2
                     Cybersecurity_Career_Opportunities       1.00      0.33      0.50         3
                           Cybersecurity_Certifications       1.00      1.00      1.00         1
                       Cybersecurity_Course_Information       0.00      0.00      0.00         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                                    Data_Science_Career       0.00      0.00      0.00         2
                      Data_Science_Career_Opportunitie

The TF-IDF + SVM model achieved an overall accuracy of approximately 39.7% on the text classification task. While this may seem modest, it's essential to delve into the classification report for deeper insights.

**Insights from the Classification Report:**
- **Precision, Recall, and F1-Score:** The model's performance varies across different classes. For instance, categories like "Cybersecurity_Certifications" and "Product_Design_UI/UX_Course_Information" exhibit high precision, recall, and F1-scores, suggesting accurate predictions. In contrast, classes such as "Data_Science_Application_Process" and "Data_Science_Courses" show lower scores, indicating challenges in correct classification.

**Confusion Matrix Insights:**
- **Pattern Recognition:** The confusion matrix provides a pattern of true positive, true negative, false positive, and false negative predictions for each class. Observing these patterns can offer insights into specific challenges the model faces. For instance, some classes may have more confusion with others, pointing to potential similarities in the underlying text.

**Overall Observation:**
- The model's strength appears to vary across different topics, reflecting the complexity of text classification. Fine-tuning or adjusting the model hyperparameters might enhance performance, especially for classes where precision, recall, or F1-score are suboptimal.

It's crucial to iteratively refine the model, considering the nature of the data and specific challenges posed by certain classes. Additionally, exploring more advanced techniques or incorporating domain-specific features could further enhance the model's effectiveness.

### Cross-validation with the TF-IDF + SVM 

In [11]:
# Cross-validation with the TF-IDF + SVM pipeline
cross_val_scores = cross_val_score(pipeline, X_train_str, y_train, cv=5)  # Adjust the number of folds as needed
average_cross_val_accuracy = np.mean(cross_val_scores)

print("Average Cross-Validation Accuracy:", average_cross_val_accuracy)

# Cross-validated predictions
cross_val_predictions = cross_val_predict(pipeline, X_train_str, y_train, cv=5)

# Classification report for cross-validated predictions
print("Cross-Validated Classification Report:\n", classification_report(y_train, cross_val_predictions))

# Confusion matrix for cross-validated predictions
conf_matrix_cross_val = confusion_matrix(y_train, cross_val_predictions)
print("Cross-Validated Confusion Matrix:\n", conf_matrix_cross_val)

Average Cross-Validation Accuracy: 0.3954945054945055
Cross-Validated Classification Report:
                                                          precision    recall  f1-score   support

                           Advanced_Data_Science_Topics       0.67      0.40      0.50         5
                                                Contact       1.00      0.50      0.67         8
                     Cybersecurity_Career_Opportunities       0.00      0.00      0.00         6
                           Cybersecurity_Certifications       0.75      0.86      0.80         7
                       Cybersecurity_Course_Information       0.60      0.43      0.50         7
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         3
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         6
                       Data_Science_Application_Process       0.00      0.00      0.00         3
                  Data_Science_A

The average cross-validation accuracy of the TF-IDF + SVM model is approximately 39.55%. Let's analyze the results in the requested format:

1. **High Precision, Low Recall:**
   - `Cybersecurity_Certifications`: Precision 75%, Recall 86%
   - `Data_Science_Career_Opportunities`: Precision 9%, Recall 27%
   - `Data_Science_Online_and_Offline_Learning_Modes`: Precision and Recall both 0%
   - `Software_Engineering_Mobile_Track`: Precision and Recall both 0%
   - `Financial Aid`: Precision and Recall both 0%
   - `cloud_computing`: Precision and Recall both 0%
   - `cloud_developer_role`: Precision and Recall both 0%
   - `hr_data_analytics`: Precision and Recall both 0%
   - `aws_cloud_course`, `aws_cloud_practitioner`, `career_services`, `cloud_computing`, `cloud_developer_role`: Precision and Recall all 0%

2. **Other Classes:**
   - `Advanced_Data_Science_Topics`: Precision 67%, Recall 40%, F1-score 50%
   - `Contact`: Precision 100%, Recall 50%, F1-score 67%
   - `Cybersecurity_Career_Opportunities`: Precision 0%, Recall 0%, F1-score 0%
   - `Cybersecurity_Course_Information`: Precision 60%, Recall 43%, F1-score 50%
   - `Data_Ethics_and_Privacy`: Precision and Recall both 0%
   - `Data_Science_Admission_Eligibility_Criteria`: Precision and Recall both 0%
   - `Data_Science_Application_Process`: Precision and Recall both 0%
   - `Data_Science_Applications_in_Business`: Precision 80%, Recall 80%, F1-score 80%
   - `Data_Science_Career`: Precision and Recall both 0%
   - `Data_Science_Career_Opportunities`: Precision 9%, Recall 27%, F1-score 13%
   - `Data_Science_Career_Transition_Support`: Precision and Recall both 0%
   - `Data_Science_Certification_and_Accreditation`: Precision and Recall both 0%
   - `Data_Science_Collaborative_Projects_and_Teamwork`: Precision and Recall both 0%
   - `Data_Science_Competitions_and_Challenges`: Precision and Recall both 0%
   - `Data_Science_Course_Duration_and_Fees`: Precision 20%, Recall 25%, F1-score 22%
   - `Data_Science_Course_Overview`: Precision 29%, Recall 29%, F1-score 29%
   - `Data_Science_Course_Pacing_and_Flexibility`: Precision and Recall both 0%
   - `Data_Science_Course_Schedule_and_Structure`: Precision 9%, Recall 22%, F1-score 13%
   - `Data_Science_Courses`: Precision and Recall both 0%
   - `Data_Science_Curriculum_Outline`: Precision 23%, Recall 58%, F1-score 33%
   - `Data_Science_Cybersecurity_Introduction`: Precision 67%, Recall 40%, F1-score 50%
   - `Data_Science_Industry_Relevance`: Precision and Recall both 0%
   - `Data_Science_Networking_and_Collaboration_Opportunities`: Precision and Recall both 0%
   - `Data_Science_Online_and_Offline_Learning_Modes`: Precision and Recall both 0%
   - `Data_Science_Pre-Work_Requirements`: Precision and Recall both 0%
   - `Data_Science_Project_and_Hands-on_Experience`: Precision 50%, Recall 78%, F1-score 61%
   - `Data_Science_Skill_Development`: Precision 6%, Recall 10%, F1-score 7%
   - `Data_Science_Software_Engineering_Fundamentals`: Precision and Recall both 0%
   - `Data_Science_Statistics_and_Analysis`: Precision 100%, Recall 89%, F1-score 94%
   - `Data_Science_Student_Support_Services`: Precision and Recall both 0%
   - `DevOps_Admissions_and_Fees`: Precision and Recall both 0%
   - `DevOps_Career_Opportunities`: Precision and Recall both 0%
   - `DevOps_Course_Information`: Precision 57%, Recall 73%, F1-score 64%
   - `Enrollment_Deadlines`: Precision and Recall both 0%
   - `Enrollment_Information`: Precision 58%, Recall 70%, F1-score 64%
   - `Ethical_Considerations_in_Data_Science`: Precision and Recall both 0%
   - `Event`: Precision 100%, Recall 100%, F1-score 100%
   - `Financial Aid`: Precision 36%, Recall 57%, F1-score 44%
   - `Full_Stack_Software_Engineering`: Precision 57%, Recall 50%, F1-score 53%
   - `Instructor_Expertise_and_Background`: Precision and Recall both 0%
   - `International_Student_Information`: Precision and Recall both 0%
   - `Mobile Development`: Precision and Recall both 0%
   - `Mobile_Development_Career`: Precision 75%, Recall 43%, F1-score 55%
   - `Mobile_Development_Courses`: Precision 60%, Recall 75%, F1-score 67%
   - `Moringa_School_Courses`: Precision and Recall both 0%
   - `Moringa_School_News`: Precision 43%, Recall 38%, F1-score 40%
   - `Moringa_School_Testimonials`: Precision 80%, Recall 67%, F1-score 73%
   - `Mpesa_Mini_Apps_API_Course_Information`: Precision 33%, Recall 50%, F1-score 40%
   - `Part-time_Course_Content`: Precision and Recall both 0%
   - `Part-time_Software Engineering`: Precision and Recall both 0%
   - `Partnership`: Precision 71%, Recall 83%, F1-score 77%
   - `Product_Design_UI/UX_Career_Opportunities`: Precision 89%, Recall 80%, F1-score 84%
   - `Product_Design_UI/UX_Course_Information`: Precision 75%, Recall 75%, F1-score 75%
   - `Software_Engineering_Course_Content`: Precision and Recall both 0%
   - `Software_Engineering_Mobile_Track`: Precision 64%, Recall 88%, F1-score 74%
   - `Student_Life_and_Break_Periods`: Precision and Recall both 0%
   - `ai_integration`: Precision 89%, Recall 89%, F1-score 89%
   - `aws_cloud_course`: Precision 90%, Recall 100%, F1-score

 95%
   - `aws_cloud_practitioner`: Precision 100%, Recall 67%, F1-score 80%
   - `career_services`: Precision 29%, Recall 29%, F1-score 29%
   - `cloud_computing`: Precision 100%, Recall 88%, F1-score 93%
   - `cloud_developer_role`: Precision 100%, Recall 100%, F1-score 100%
   - `course_details_Front-End_web_development`: Precision 62%, Recall 56%, F1-score 59%
   - `detailed_schedule`: Precision 50%, Recall 25%, F1-score 33%
   - `enrollment`: Precision 25%, Recall 12%, F1-score 17%
   - `enrollment_deadline`: Precision 43%, Recall 38%, F1-score 40%
   - `enrollment_information`: Precision 12%, Recall 22%, F1-score 16%
   - `hr_data_analytics`: Precision 83%, Recall 62%, F1-score 71%
   - `interview_process`: Precision 90%, Recall 100%, F1-score 95%
   - `learning_resources`: Precision and Recall both 0%
   - `online_learning`: Precision 55%, Recall 86%, F1-score 67%
   - `professional_advancement`: Precision 9%, Recall 11%, F1-score 10%
   - `student_community`: Precision and Recall both 0%
   - `tech_based_learning`: Precision and Recall both 0%

In summary, the model's performance across classes is diverse. Precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides insights into correct and incorrect predictions for each class, aiding in identifying areas for improvement. Further refinement through hyperparameter tuning or additional data may enhance overall performance.

### GridSearchCV for SVM

In [12]:
# Define the parameter grid for SVM
svm_param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10, 100]}

# Create a GridSearchCV object for SVM
svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=5)
svm_grid_search.fit(X_train_vectorized, y_train)

# Get the best hyperparameters for SVM
best_svm_params = svm_grid_search.best_params_

# Train the SVM model with the best hyperparameters
best_svm_model = SVC(**best_svm_params)
best_svm_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the best SVM model
best_svm_predictions = best_svm_model.predict(X_test_vectorized)

# Evaluate the best SVM model
best_svm_accuracy = accuracy_score(y_test, best_svm_predictions)
print("Best SVM Model Accuracy:", best_svm_accuracy)

# Classification report for the best SVM model
best_svm_classification_report = classification_report(y_test, best_svm_predictions)
print("\nBest SVM Model Classification Report:\n", best_svm_classification_report)

# Confusion matrix for the best SVM model
best_svm_confusion_matrix = confusion_matrix(y_test, best_svm_predictions)
print("\nBest SVM Model Confusion Matrix:\n", best_svm_confusion_matrix)

# Cross-validation with the best SVM model
best_svm_cross_val_scores = cross_val_score(best_svm_model, X_train_vectorized, y_train, cv=5)
average_best_svm_cross_val_accuracy = np.mean(best_svm_cross_val_scores)
print("\nAverage Cross-Validation Accuracy for Best SVM Model:", average_best_svm_cross_val_accuracy)

Best SVM Model Accuracy: 0.3893129770992366

Best SVM Model Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       1.00      0.50      0.67         2
                     Cybersecurity_Career_Opportunities       1.00      0.33      0.50         3
                           Cybersecurity_Certifications       1.00      1.00      1.00         1
                       Cybersecurity_Course_Information       1.00      0.33      0.50         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                  Data_Science_Applications_in_Business       0.00      0.00      0.00         0
                                    Data_S

The average cross-validation accuracy of the TF-IDF + SVM model is approximately 38.93%. Let's analyze the results in the requested format:

**High Precision, Low Recall:**
- Cybersecurity_Certifications: Precision 100%, Recall 100%
- Data_Science_Project_and_Hands-on_Experience: Precision 50%, Recall 100%
- ai_integration, aws_cloud_course, aws_cloud_practitioner, career_services, cloud_computing, cloud_developer_role: Precision and Recall all 100%

**Other Classes:**
- Contact: Precision 100%, Recall 50%
- Cybersecurity_Career_Opportunities: Precision 100%, Recall 33%
- Cybersecurity_Course_Information: Precision 100%, Recall 33%
- Data_Science_Career: Precision 25%, Recall 50%
- Data_Science_Course_Overview: Precision 60%, Recall 60%
- Data_Science_Online_and_Offline_Learning_Modes: Precision 100%, Recall 50%
- DevOps_Career_Opportunities: Precision 50%, Recall 50%
- DevOps_Course_Information: Precision 40%, Recall 100%
- Event: Precision 100%, Recall 100%
- Financial Aid: Precision 0%, Recall 0%
- Full_Stack_Software_Engineering: Precision 33%, Recall 33%
- Mobile_Development_Career: Precision 50%, Recall 33%
- Mobile_Development_Courses: Precision 33%, Recall 100%
- Moringa_School_News: Precision 67%, Recall 100%
- Moringa_School_Testimonials: Precision 100%, Recall 25%
- Part-time_Course_Content: Precision and Recall both 0%
- Part-time_Software_Engineering: Precision and Recall both 0%
- Partnership: Precision 50%, Recall 50%
- Product_Design_UI/UX_Career_Opportunities: Precision and Recall both 0%
- Product_Design_UI/UX_Course_Information: Precision 100%, Recall 100%
- Software_Engineering_Course_Content: Precision and Recall both 0%
- Software_Engineering_Mobile_Track: Precision and Recall both 0%
- ai_integration: Precision 100%, Recall 100%
- aws_cloud_course: Precision 100%, Recall 100%
- aws_cloud_practitioner: Precision 100%, Recall 100%
- career_services: Precision 75%, Recall 100%
- cloud_computing: Precision 100%, Recall 100%
- cloud_developer_role: Precision 100%, Recall 100%
- detailed_schedule: Precision 50%, Recall 50%
- enrollment: Precision 0%, Recall 0%
- enrollment_deadline: Precision 20%, Recall 100%
- enrollment_information: Precision 14%, Recall 100%
- hr_data_analytics: Precision 100%, Recall 100%
- interview_process: Precision 100%, Recall 100%
- learning_resources: Precision 100%, Recall 20%
- online_learning: Precision 100%, Recall 67%
- student_community: Precision 50%, Recall 50%
- tech_based_learning: Precision 50%, Recall 33%

In summary, the model's performance across classes is diverse. Precision measures the accuracy of positive predictions, while recall measures the coverage of actual positive instances. The confusion matrix provides insights into correct and incorrect predictions for each class, aiding in identifying areas for improvement. Further refinement through hyperparameter tuning or additional data may enhance overall performance.

### RandomForestClassifier

In [13]:
# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the Random Forest model
rf_predictions = rf_model.predict(X_test_vectorized)

# Classification report for the Random Forest model
rf_classification_report = classification_report(y_test, rf_predictions)

# Confusion matrix for the Random Forest model
rf_confusion_matrix = confusion_matrix(y_test, rf_predictions)

# Cross-validation with the Random Forest model
rf_cross_val_scores = cross_val_score(rf_model, X_train_vectorized, y_train, cv=5)
average_rf_cross_val_accuracy = np.mean(rf_cross_val_scores)

print("Random Forest Model Classification Report:\n", rf_classification_report)
print("\nRandom Forest Model Confusion Matrix:\n", rf_confusion_matrix)
print("\nAverage Cross-Validation Accuracy for Random Forest Model:", average_rf_cross_val_accuracy)

Random Forest Model Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       1.00      1.00      1.00         2
                     Cybersecurity_Career_Opportunities       1.00      0.67      0.80         3
                           Cybersecurity_Certifications       1.00      1.00      1.00         1
                       Cybersecurity_Course_Information       1.00      0.33      0.50         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                  Data_Science_Applications_in_Business       0.00      0.00      0.00         0
                                    Data_Science_Career       0.33      0.50      

The Random Forest Model's classification report provides metrics such as precision, recall, and F1-score for each class. Here are key insights:

- **Precision:** Indicates the accuracy of positive predictions.
- **Recall:** Reflects the ability of the model to capture all positive instances.
- **F1-score:** Harmonic mean of precision and recall, providing a balanced measure.

**Notable observations:**
- The model performs well for certain classes, achieving high precision, recall, and F1-score (e.g., "Contact," "Cybersecurity_Certifications," and "Event").
- Some classes have lower performance, with zero precision, recall, and F1-score. This may be due to insufficient data or the complexity of distinguishing those classes.
- The overall accuracy of the model is 44%, suggesting room for improvement.

**Confusion Matrix:**
- Each row represents the actual class, and each column represents the predicted class.
- Diagonal elements indicate correct predictions, while off-diagonal elements represent misclassifications.

**Average Cross-Validation Accuracy:**
- The average accuracy from cross-validation is approximately 41.7%, giving an estimate of the model's generalization performance.

**Conclusion:**
- The model shows promising performance for certain classes but may benefit from further refinement, especially for classes with zero precision and recall. Fine-tuning the model or addressing class imbalances could enhance overall accuracy.

### GridSearchCV for Random Forest Classifier

In [14]:
# Define the parameter grid to search for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]  # Include valid options for max_features
}

# Create a GridSearchCV object for Random Forest
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5)
grid_search_rf.fit(X_train_vectorized, y_train)

# Get the best hyperparameters for Random Forest
best_params_rf = grid_search_rf.best_params_

# Train the Random Forest model with the best hyperparameters
best_model_rf = RandomForestClassifier(**best_params_rf)
best_model_rf.fit(X_train_vectorized, y_train)

# Predictions on the test set using the best Random Forest model
best_rf_predictions = best_model_rf.predict(X_test_vectorized)

# Classification report for the best Random Forest model
best_rf_classification_report = classification_report(y_test, best_rf_predictions)

# Confusion matrix for the best Random Forest model
best_rf_confusion_matrix = confusion_matrix(y_test, best_rf_predictions)

# Cross-validation with the best Random Forest model
best_rf_cross_val_scores = cross_val_score(best_model_rf, X_train_vectorized, y_train, cv=5)
average_best_rf_cross_val_accuracy = np.mean(best_rf_cross_val_scores)

print("Best Random Forest Model Classification Report:\n", best_rf_classification_report)
print("\nBest Random Forest Model Confusion Matrix:\n", best_rf_confusion_matrix)
print("\nAverage Cross-Validation Accuracy for Best Random Forest Model:", average_best_rf_cross_val_accuracy)


Best Random Forest Model Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       1.00      0.50      0.67         2
                     Cybersecurity_Career_Opportunities       0.00      0.00      0.00         3
                           Cybersecurity_Certifications       1.00      1.00      1.00         1
                       Cybersecurity_Course_Information       0.33      0.33      0.33         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                  Data_Science_Applications_in_Business       0.00      0.00      0.00         0
                                    Data_Science_Career       0.00      0.00 

In [15]:
# Get the best hyperparameters for Random Forest
best_params_rf

{'max_depth': 30,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 200}

The best Random Forest model's classification report provides insights into its performance on various classes. Here are some key observations:

1. **Precision, Recall, and F1-Score:** Precision represents the accuracy of positive predictions, recall measures the ability to capture all positive instances, and F1-score balances both. The results vary across classes.

2. **Class-Specific Insights:**
   - Classes like "Contact," "Cybersecurity_Certifications," and "Event" have perfect precision, recall, and F1-scores, indicating accurate predictions.
   - Some classes, such as "Data_Ethics_and_Privacy" and "Data_Science_Admission_Eligibility_Criteria," have low or zero scores, suggesting challenges in prediction.

3. **Average Metrics:**
   - The overall accuracy of the model is 46%, indicating its ability to correctly classify instances.
   - Macro and weighted averages for precision, recall, and F1-score provide a holistic view of the model's performance across all classes.

4. **Confusion Matrix:**
   - The confusion matrix breaks down the actual vs. predicted counts for each class.
   - Diagonal elements represent correct predictions, while off-diagonal elements indicate misclassifications.

5. **Cross-Validation Accuracy:**
   - The average cross-validation accuracy is around 44%, showing consistency in performance across different data splits.

6. **Improvements:**
   - While the model performs well in some classes, there is room for improvement in others.
   - Further optimization or exploration of data features may enhance overall performance.

### XGBoost model

In [16]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Create an XGBoost model
xgb_model = XGBClassifier()

# Train the XGBoost model
xgb_model.fit(X_train_vectorized, y_train)

# Predictions on the test set using the XGBoost model
xgb_predictions = xgb_model.predict(X_test_vectorized)

# Classification report for the XGBoost model
xgb_classification_report = classification_report(y_test, xgb_predictions)

# Confusion matrix for the XGBoost model
xgb_confusion_matrix = confusion_matrix(y_test, xgb_predictions)

# Accuracy for the XGBoost model
xgb_accuracy = accuracy_score(y_test, xgb_predictions)

print("XGBoost Model Classification Report:\n", xgb_classification_report)
print("\nXGBoost Model Confusion Matrix:\n", xgb_confusion_matrix)
print("\nAccuracy for XGBoost Model:", xgb_accuracy)

XGBoost Model Classification Report:
                                                          precision    recall  f1-score   support

                                                Contact       0.00      0.00      0.00         2
                     Cybersecurity_Career_Opportunities       1.00      0.33      0.50         3
                           Cybersecurity_Certifications       0.50      1.00      0.67         1
                       Cybersecurity_Course_Information       0.50      0.33      0.40         3
                                Data_Ethics_and_Privacy       0.00      0.00      0.00         2
            Data_Science_Admission_Eligibility_Criteria       0.00      0.00      0.00         1
                       Data_Science_Application_Process       0.00      0.00      0.00         2
                                    Data_Science_Career       0.00      0.00      0.00         2
                      Data_Science_Career_Opportunities       0.00      0.00      0.00  

The XGBoost model achieved an accuracy of 33.59%. Here are some key performance metrics for specific classes:

- **High Performance (Precision, Recall, F1-Score):**
  - 'Event': 100% precision, recall, and F1-score.
  - 'Mpesa_Mini_Apps_API_Course_Information', 'aws_cloud_course', 'aws_cloud_practitioner', 'cloud_computing', 'cloud_developer_role', 'Software_Engineering_Mobile_Track', 'ai_integration': All achieved perfect scores.

- **Low Performance (Precision, Recall, F1-Score):**
  - Several classes, such as 'Data_Science_Career', 'Data_Science_Courses', 'Data_Science_Industry_Relevance', 'Data_Science_Networking_and_Collaboration_Opportunities', and 'learning_resources', showed low scores, indicating room for improvement.

- **Overall Observation:**
  - The model struggles with certain classes, potentially requiring further tuning or additional data for those categories.

# Conclusion

Among the models discussed, the Random Forest model stands out as one of the best performers. It shows promising results with high precision, recall, and F1-scores for certain classes, such as "Contact," "Cybersecurity_Certifications," and "Event." The overall accuracy of the Random Forest model is 44%, suggesting its ability to correctly classify instances. However, it is noted that there is room for improvement, especially for classes with zero precision and recall, indicating potential challenges in distinguishing those classes.

On the other hand, the TF-IDF + SVM model appears to have a diverse performance across different classes. While some classes show high precision and recall, others, particularly "Learning_Resources," exhibit low performance with precision, recall, and F1-score all at 0%. The average cross-validation accuracy for this model ranges from approximately 38.93% to 39.7%.

In summary, the Random Forest model emerges as one of the best-performing models in terms of precision, recall, and overall accuracy, while the TF-IDF + SVM model demonstrates diversity in performance, with certain classes requiring improvement. Fine-tuning and further exploration of data features are recommended for both models, with the Random Forest model showing particular promise for refinement.